In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import torch.cuda.amp as amp
from tqdm.notebook import tqdm
import numpy as np
import json


class LocalImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Get class names and create label mapping
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        # Collect all image paths and their labels
        self.images = []
        self.labels = []
        
        for cls_name in self.classes:
            class_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(self.class_to_idx[cls_name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_test_split(dataset, test_size=0.2, shuffle=True):
    """
    Split dataset into train and test sets
    
    Args:
    - dataset: The full dataset
    - test_size: Proportion of dataset to include in test split (0.0 to 1.0)
    - shuffle: Whether to shuffle before splitting
    
    Returns:
    - train_dataset, test_dataset
    """
    test_size = int(len(dataset) * test_size)
    train_size = len(dataset) - test_size
    
    return random_split(dataset, [train_size, test_size])


def evaluate_model(model, test_dataloader, device, output_file="evaluation_results_resnet_no defect.json"):
    """
    Evaluate the model on test dataset and save the results to a JSON file.
    
    Args:
    - model: Trained PyTorch model
    - test_dataloader: DataLoader for test dataset
    - device: torch.device to run the evaluation on
    - output_file: Path to save the output JSON file
    
    Returns:
    - Dictionary of evaluation metrics
    """
    model.eval()
    correct = 0
    total = 0
    class_correct = [0] * len(test_dataloader.dataset.dataset.classes)
    class_total = [0] * len(test_dataloader.dataset.dataset.classes)
    
    confusion_matrix = np.zeros((len(test_dataloader.dataset.dataset.classes), 
                                  len(test_dataloader.dataset.dataset.classes)), 
                                 dtype=int)
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_dataloader, desc="Evaluation", position=0):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Per-class accuracy
            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                class_correct[label] += (pred == label).item()
                class_total[label] += 1
                
                # Confusion matrix
                confusion_matrix[label][pred] += 1
    
    # Calculate metrics
    overall_accuracy = 100 * correct / total
    
    # Per-class accuracy
    class_accuracies = [100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0 
                        for i in range(len(class_total))]
    
    # Prepare the results dictionary
    results = {
        'overall_accuracy': overall_accuracy,
        'class_accuracies': class_accuracies,
        'confusion_matrix': confusion_matrix.tolist(),  # Convert numpy array to list for JSON serialization
        'class_names': test_dataloader.dataset.dataset.classes
    }
    
    try:
        with open(output_file, 'r') as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        data = []

    # Append the new epoch data to the list
    data.append(results)

    # Write the updated data back to the file
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)
    return results

def fine_tune_model(
    dataset_path, 
    batch_size=8, 
    num_epochs=10, 
    learning_rate=0.0001,
    test_size=0.2
):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        #transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # Create full dataset
    full_dataset = LocalImageDataset(dataset_path, transform=transform)
    
    # Split dataset
    train_dataset, test_dataset = train_test_split(full_dataset, test_size=test_size)
    
    # Create DataLoaders
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    test_dataloader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    # Get number of classes
    num_classes = len(full_dataset.classes)
    print(f"Number of classes detected: {num_classes}")
    print("Classes:", full_dataset.classes)
    print(f"Total images: {len(full_dataset)}")
    print(f"Training images: {len(train_dataset)}")
    print(f"Test images: {len(test_dataset)}")
    
    # Load pre-trained model
    model = models.resnet50(pretrained=True)
    # Modify input layer and final classification layer
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = torch.nn.DataParallel(model)

    # Move to GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Mixed precision training
    scaler = amp.GradScaler()
    
    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0):
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        
        # Progress bar for batches
        batch_progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", position=1, leave=False)
        
        for inputs, labels in batch_progress:
            # Move to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Mixed precision forward pass
            with amp.autocast(enabled=True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            # Backward pass
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Update batch progress bar
            batch_progress.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Accuracy': f"{correct_predictions / total_samples * 100:.2f}%"
            })
        
        # Print epoch statistics
        epoch_loss = total_loss / len(train_dataloader)
        epoch_accuracy = correct_predictions / total_samples * 100

        # Prepare the data to append
        epoch_data = {
            "epoch": epoch + 1,
            "loss": epoch_loss,
            "accuracy": epoch_accuracy
        }

        # Append to the JSON file
        json_file = 'train_eval_no defct.json'

        # Load existing data from the file, if it exists, or initialize as an empty list
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            data = []

        # Append the new epoch data to the list
        data.append(epoch_data)

        # Write the updated data back to the file
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)

        # Print to console (optional, as you want to append to JSON, not print)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    # Evaluate the model
        eval_results = evaluate_model(model, test_dataloader, device)
    
    # Print evaluation results
        print("\nEvaluation Results:")
        print(f"Overall Accuracy: {eval_results}%")
    
    print("\nClass-wise Accuracies:")
    for cls, acc in zip(eval_results['class_names'], eval_results['class_accuracies']):
        print(f"{cls}: {acc:.2f}%")
    
    # Save the model with evaluation results
    save_path = 'fine_tuned_model_NoDefect.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_to_idx': full_dataset.class_to_idx,
        'classes': full_dataset.classes,
        'evaluation_results': eval_results
    }, save_path)
    print(f"\nModel saved to {save_path}")
    
    return model, eval_results



In [2]:
model, eval_results= fine_tune_model('dataset_final', num_epochs=100,batch_size=16)


Number of classes detected: 8
Classes: ['.ipynb_checkpoints', 'Core deformation', 'Core displacement', 'Core split', 'Foreign Object Damage', 'No defect', 'Resin buildup', 'Splice gap']
Total images: 3350
Training images: 2680
Test images: 670


/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_1829/3341243111.py:198: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/168 [00:00<?, ?it/s]

/tmp/ipykernel_1829/3341243111.py:219: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=True):


...

Epoch 21/100 - Loss: 0.1993, Accuracy: 94.66%


Evaluation:   0%|          | 0/42 [00:00<?, ?it/s]


Evaluation Results:
Overall Accuracy: {'overall_accuracy': 72.68656716417911, 'class_accuracies': [0, 69.66824644549763, 73.33333333333333, 82.22222222222223, 86.07594936708861, 43.75, 56.52173913043478, 76.47058823529412], 'confusion_matrix': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 147, 2, 21, 28, 2, 4, 7], [0, 11, 143, 2, 34, 2, 2, 1], [0, 1, 2, 37, 0, 0, 2, 3], [0, 2, 3, 1, 68, 2, 2, 1], [0, 8, 3, 0, 4, 14, 2, 1], [0, 1, 1, 1, 6, 0, 13, 1], [0, 5, 3, 1, 6, 4, 1, 65]], 'class_names': ['.ipynb_checkpoints', 'Core deformation', 'Core displacement', 'Core split', 'Foreign Object Damage', 'No defect', 'Resin buildup', 'Splice gap']}%


Epoch 22:   0%|          | 0/168 [00:00<?, ?it/s]

KeyboardInterrupt: 